In [32]:
import spotipy
from spotipy import oauth2

from difflib import SequenceMatcher

In [7]:
path = '../data/composers.csv'

composers = []
with open(path, 'r') as f:
    next(f)
    for c_id, line in enumerate(f):
        c = line.strip().split(',')
        composers.append((c_id, *c))
        
composers_by_id = {c[0]: c[1:] for c in composers}

print(composers_by_id[0])

('Mary Anne à Beckett', '1817', '1863', 'https://en.wikipedia.org/wiki/Mary_Anne_%C3%A0_Beckett')


In [42]:
client_credentials_manager = oauth2.SpotifyClientCredentials(
    client_id='ee7f5e0be5b84819af40934d69c20547',
    client_secret='c819b920aa3b425ba8bdf100e3923eb9')
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

spotify_id_by_composer_id = {}
spotify_name_by_composer_id = {}

for c_id, composer in composers_by_id.items():
    target_composer_name = composer[0]
    
    results = spotify.search(target_composer_name, type='artist')
    artists = results['artists']['items']
    
    if not artists:
        spotify_id_by_composer_id[c_id] = None
        spotify_name_by_composer_id[c_id] = None
        continue

    best_match = max(artists, key=lambda a: similar(a['name'], target_composer_name))
    spotify_id_by_composer_id[c_id] = best_match['id']
    spotify_name_by_composer_id[c_id] = best_match['name']


retrying ...1secs


In [62]:
composer_id_by_spotify_name = {v: k for k, v in spotify_name_by_composer_id.items()}

related_composer_ids_by_composer_id = {}

for c_id, spotify_id in spotify_id_by_composer_id.items():
    if not spotify_id:
        related_composer_ids_by_composer_id[c_id] = None
        continue
        
    spotify_related_artists = spotify.artist_related_artists(spotify_id)['artists']
    related_composer_ids = [c_id for c_id in [composer_id_by_spotify_name.get(artist['name']) for artist in spotify_related_artists] if c_id]
    related_composer_ids_by_composer_id[c_id] = ':'.join(map(str, related_composer_ids))

print(related_composer_ids_by_composer_id)

{0: None, 1: '1586:2241:1476:3007:1038:508:1713:1431', 2: '', 3: '2587:1981:2300:2398:2412', 4: None, 5: None, 6: None, 7: '', 8: '', 9: None, 10: '1816:1826:2330:2020:3450:1647:2964:900:555:1817:639:314', 11: '1071:575:2804', 12: None, 13: '2373:547:1036', 14: '', 15: '', 16: None, 17: '326:3519', 18: '', 19: '', 20: '', 21: '1203:2788:1338:3340:2009:2700:270:485', 22: '', 23: None, 24: None, 25: None, 26: '2520', 27: '2376:2378:3225:2834', 28: '', 29: None, 30: None, 31: '', 32: '1315:3511:1909:3370:1713:1833:442:1495', 33: '2424:434:2004', 34: '', 35: '1966:1142:2737:2401:2930:608:633:2120:3164:1165:472:2131:1936:1696:2703:764', 36: '547:2061:2085:1301:803:63:3033', 37: '', 38: '', 39: '', 40: '', 41: None, 42: '', 43: '2530:3257:1897:552:2906:1773', 44: '2530:3257:1897:552:2906:1773', 45: '', 46: '', 47: None, 48: '', 49: '', 50: '2030:195:296:1291:391:2299:1736:2543:1737:1666:3428', 51: '1176:679', 52: '1927:1879:2427:2278:3466:2273:2744', 53: None, 54: None, 55: None, 56: '156:26

In [68]:
target_path = '../data/composers-spotify.csv'

with open(target_path, 'w') as f_w:
    with open(path, 'r') as f:
        f_w.write('name,born,died,url,spotify_id,spotify_related_artist_ids')
        next(f)
        for c_id, line in enumerate(f):
            spotify_id = spotify_id_by_composer_id[c_id] or ''
            related_artist_ids = related_composer_ids_by_composer_id[c_id] or ''
            f_w.write(','.join((line.strip(), spotify_id, related_artist_ids)))
            f_w.write('\n')